---

# CSCI 3202, Fall 2022
# Final Project
# Project Due: Thursday December 8, 2022 at 6:00 PM
## Proposals Due: Friday November 18, 2022 at 6:00 PM


You have two options for completing your final project for this course. 

#### Option 1 ####
`The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. `


**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- Send an email to the course instructors before Friday, November 18 at 6pm, with a paragraph description of your project. We will respond within 24 hours with feedback.
- The project can include an algorithm we've discussed throughout the semester or an algorithm that you're been curious to learn. Please don't recycle a project that you did in another class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of an exam. So, we will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If any part of the given project or your personal project is left open-ended, it is because we intend for you to code it up however you want. Our primary concern is with the plots/analysis that your code produces. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

**Your name:** `Adrian Ornelas Ruvalcaba`

---


---

Some useful packages and libraries:



In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [43]:
class State:
    def __init__(self, moves):
        self.to_move = 'R'
        self.utility = 0
        self.board = {}
        self.moves = moves

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)

        self.winning_cordinates = {1:[], 2:[], 3:[], 4:[]}
        self.print_star_help = []
    
    # used from the game solution inclass notebook 
    def result(self, move, state):
        # Don't do anything if the move isn't a legal one
        if move not in state.moves:
            return state
        
        # Return a copy of the updated state:
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('B' if state.to_move == 'R' else 'R')
        return new_state

    # used from the game solution inclass notebook 
    def compute_utility(self, move, state):      
        # your code goes here...
        row, col = move
        player = state.to_move
        num_to_win = self.nwin # 6
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player
        
        #check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player

            if(in_a_row == 1):
                in_a_row = 0
                for x in range(num_to_win):
                    if(board.get((row,c+x)) == player):
                        in_a_row += 1
                    else:
                        in_a_row = 0

                if(in_a_row == num_to_win):
                    for i in range(num_to_win):
                        self.winning_cordinates[1].append((row,c+i))
                    break
                else:
                    self.winning_cordinates[1].clear()
                    in_a_row = 0                    

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player

            if(in_a_col == 1):
                in_a_col = 0
                for x in range(num_to_win):
                    if(board.get((r-x, col)) == player):
                        in_a_col += 1
                    else:
                        in_a_col += 0

                if(in_a_col == num_to_win):
                    for i in range(num_to_win):
                        self.winning_cordinates[2].append((r-i,col))
                    break
                else:
                    self.winning_cordinates[2].clear()
                    in_a_col = 0

        # check left and right diaginal win
        in_a_diag1 = 0
        in_a_diag2 = 0

        for r in range(1, self.nrow+1):
            for c in range(1,self.ncol+1):
                in_a_diag1 += board.get((r,c)) == player
                in_a_diag2 += board.get((r,c)) == player

                # check left win
                if(in_a_diag1 == 1):
                    in_a_diag1 = 0
                    for x in range(num_to_win):
                        if(board.get((r-x, c-x)) == player):
                            in_a_diag1 += 1
                        else: 
                            in_a_diag1 = 0

                    if(in_a_diag1 == num_to_win):
                        for i in range(num_to_win):
                            self.winning_cordinates[3].append((r-i, c-i))
                        break
                    else:
                        self.winning_cordinates[3].clear()
                        in_a_diag1 = 0

                # check right win
                if(in_a_diag2 == 1):
                    in_a_diag2 = 0
                    for x in range(num_to_win):
                        if(board.get((r-x, c+x)) == player):
                            in_a_diag2 += 1
                        else: 
                            in_a_diag2 = 0

                    if(in_a_diag2 == num_to_win):
                        for i in range(num_to_win):
                            self.winning_cordinates[3].append((r-i, c+i))
                        break
                    else:
                        self.winning_cordinates[3].clear()
                        in_a_diag2 = 0

        temp = [in_a_row, in_a_col, in_a_diag1, in_a_diag2]

        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            self.print_star_help = self.winning_cordinates[temp.index(self.nwin)+1]

            return 1 if player=='R' else -1
        else:
            return 0

    # used from the game solution inclass notebook 
    def game_over(self, state):
        # your code goes here...   
        return state.utility!=0 or len(state.moves)==0

    # used from the game solution inclass notebook 
    def utility(self, state, player):
        # your code goes here...
        return state.utility if player=='R' else -state.utility

    # used from the game solution inclass notebook 
    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    # used from the game solution inclass notebook but used stars to show winning patter easier, (helped debug)
    def display_winin(self):
        board_p = self.state.board
        win_cor = self.print_star_help
        print(win_cor, "winning disk locations")
        string = ''
        for r in range(1,self.nrow+1):
            for c in range(1,self.ncol+1):
                if((r,c) in win_cor):
                    string += '* '
                if(board_p.get((r,c)) == None and (r,c) not in win_cor):
                    string += '. '
                if(board_p.get((r,c)) == 'R' and (r,c) not in win_cor):
                    string += 'R '
                if(board_p.get((r,c)) == 'B' and (r,c) not in win_cor):
                    string += 'B '
            print(string)
            string = ''

    # used from the game solution inclass notebook but used stars to show winning patter easier but 
    # some how alphabeta wasnt workking correctly with other method, (helped debug)
    def display_winin_AlphaBeta(self):
        board_p = self.state.board
        win_cor = self.print_star_help[-self.nwin:]
        print(win_cor, "winning disk locations", "\n")
        string = ''
        for r in range(1,self.nrow+1):
            for c in range(1,self.ncol+1):
                if((r,c) in win_cor):
                    string += '* '
                if(board_p.get((r,c)) == None and (r,c) not in win_cor):
                    string += '. '
                if(board_p.get((r,c)) == 'R' and (r,c) not in win_cor):
                    string += 'R '
                if(board_p.get((r,c)) == 'B' and (r,c) not in win_cor):
                    string += 'B '
            print(string)
            string = ''

    # used from the game solution inclass notebook 
    def play_game(self, player1, player2, print_P=True, albe_p=False):
        # your code goes here...
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0

        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)

                # my code
                curr_row, curr_col = move
                flag = False
                # calculate free space for fall 
                for open_space in range(self.ncol, 0, -1):
                    if((curr_row, open_space) not in self.state.board):
                        new_move = (curr_row, open_space)
                        flag = True
                        break
                # if new disk had to fall 
                if(flag == True):
                    self.state = self.result(new_move, self.state)
                # else take og move
                else:
                    self.state = self.result(move, self.state)

                # if the game has ended. 
                if self.game_over(self.state):
                    if(print_P == True and albe_p == False):
                        self.display()
                        print()
                        self.display_winin()
                    elif(albe_p == True):
                        self.display()
                        print()
                        self.display_winin_AlphaBeta()
                    return self.state.utility   
                

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [44]:
# the random player is used by the the Games Solution inclass notebook 
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    
    # your code goes here...
    possible_actions = game.state.moves
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]
    


In [45]:
# now we will test 1 game
test_run = ConnectFour(6,7,4)
test_outcome = test_run.play_game(random_player, random_player)
print()
if(test_outcome == 1):
    print("Red (First Player) has won!")
elif(test_outcome == -1):
    print("Black (Second Player) has won!")
else:
    print("Its a tie!")

. . . . R B R 
. . . . B R R 
. . B R R R B 
. . . R B B R 
. . . R B B R 
. . . R B B B 

[(6, 4), (5, 4), (4, 4), (3, 4)] winning disk locations
. . . . R B R 
. . . . B R R 
. . B * R R B 
. . . * B B R 
. . . * B B R 
. . . * B B B 

Red (First Player) has won!


We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [46]:
# Your code here
game_result = {"Player1": 0, "Player2": 0, "Tie": 0}

# 1000 games between two random players
for i in range(1000):
    # create new game every run
    game_run = ConnectFour(6,7,4)
    # two random players passed in, False used to not print board 1000 times
    game_outcome = game_run.play_game(random_player, random_player, False)
    # store if player 1 won, player 2 won, or if it was a tie using dictonary
    if(game_outcome == 1):
        game_result["Player1"] += 1

    elif(game_outcome == -1):
        game_result["Player2"] += 1

    else:
        game_result["Tie"] += 1

# results 
print("Player 1 win rate:", game_result["Player1"]/1000, "\n")
print("Player 2 win rate:", game_result["Player2"]/1000, "\n")
print("Neither player won:", game_result["Tie"]/1000, "\n")

Player 1 win rate: 0.575 

Player 2 win rate: 0.409 

Neither player won: 0.016 



### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a `10x10` ConnectFour tournament, where `6` marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [47]:
# larger game board test run with larger number of back to back disks
large_game_run = ConnectFour(nrow=10,ncol=10,nwin=6)
large_game_outcome = large_game_run.play_game(random_player, random_player)
print(large_game_outcome)

. . . . . . B B B R 
. . . R B R B R R R 
. . . . . . R R B B 
. . . . . . . R B R 
. . . . . . B R B R 
. . . . . . . . B B 
. . . R R R R R R B 
. . . . . B R B B R 
. . . . . B B R R B 
. . . . . . B B R B 

[(7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9)] winning disk locations
. . . . . . B B B R 
. . . R B R B R R R 
. . . . . . R R B B 
. . . . . . . R B R 
. . . . . . B R B R 
. . . . . . . . B B 
. . . * * * * * * B 
. . . . . B R B B R 
. . . . . B B R R B 
. . . . . . B B R B 
1


In [48]:
# Your code here
large_game_result = {"Player1": 0, "Player2": 0, "Tie": 0}

# 1000 games between two random players on larger board
for i in range(1000):
    # create new game every run
    large_game_run = ConnectFour(10,10,6)
    # two random players passed in, False used to not print board 1000 times
    large_game_outcome = large_game_run.play_game(random_player, random_player, False)
    # store if player 1 won, player 2 won, or if it was a tie using dictonary
    if(large_game_outcome == 1):
        large_game_result["Player1"] += 1

    elif(large_game_outcome == -1):
        large_game_result["Player2"] += 1

    else:
        large_game_result["Tie"] += 1

# results
print("Player 1 win rate:", large_game_result["Player1"]/1000, "\n")
print("Player 2 win rate:", large_game_result["Player2"]/1000, "\n")
print("Neither player won:", large_game_result["Tie"]/1000, "\n")

Player 1 win rate: 0.409 

Player 2 win rate: 0.384 

Neither player won: 0.207 



`This is a much larger playing field, this requires a greater number of pieces to be in a row to warrent a win. Due to this, the random player is a lot less likley to be able to place 6 pieces right next to each other. This leads to the most probable outcome being the playing field get full before their can be a concluding victor. This leads to a large probability to being a tie.`

### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [49]:
# used from refrence of lecture notes 'game' also games4e.py from class repo, 
# also from my homework 7. The structure is mainly from lecture notes
def alphabeta_player(game):
    # defult values for alpha and beta
    alpha = -float('inf')
    beta = float('inf')
    # get best action and value assosicated with it
    value, best_action = max_value(game, game.state, alpha, beta)
    return best_action

# will find the max 
def max_value(game, state, alpha, beta):
    # check if game is over
    if game.game_over(state):
        # return no move because game is over and action shouldnt be performed 
        return (state.utility, None)
    # starting values 
    temp_value = -float('inf')
    temp_move = None
    # go through all possible moves possible
    for action in state.moves:
        # calculate if new values and moves
        new_value, new_move = min_value(game, game.result(action, state), alpha, beta)
        # get max of values and set the move if the move passed in had a higher value 
        temp_value, temp_move = max(temp_value, new_value), action if new_value > temp_value else temp_move
        # this is the pruning step: we return early and dont check the rest
        if temp_value >= beta:
            return (temp_value, temp_move)
        # update alpha by choosing the max and try again
        alpha = max(temp_value, alpha)
    return (temp_value, temp_move)

# will find the min
def min_value(game, state, alpha, beta):
    # check if game is over
   if game.game_over(state):
        # return no move because game is over and action shouldnt be performed
       return (state.utility, None)
   # starting values 
   temp_value = float('inf')
   temp_move = None
   # go through all possible moves possible
   for action in state.moves:
        # calculate if new values and moves
        new_value, new_move = max_value(game, game.result(action, state), alpha, beta)
        # get min of values and set the move if the move passed in had a higher value 
        temp_value, temp_move = min(temp_value, new_value), action if new_value < temp_value else temp_move
        # this is the pruning step: we return early and dont check the rest
        if temp_value <= alpha:
            return (temp_value, temp_move)
        # update beta by choosing the min and try again
        beta = min(temp_value, beta)
   return (temp_value, temp_move)

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [50]:
# Test run the game with small board, any larger the game takes a very long time and strains my computer 
print("AB pruning player vs Random Player")
game_run_ar = ConnectFour(3,3,3)
# the True statments are used to help me print the board showing the winning row
game_ar_result = game_run_ar.play_game(alphabeta_player, random_player, True, True)
print(game_ar_result)
print('------------------------------')
# Test run the game with small board, any larger the game takes a very long time and strains my computer 
print("AB pruning player vs AB pruning player")
game_run_as = ConnectFour(3,3,3)
game_as_result = game_run_as.play_game(alphabeta_player, alphabeta_player, True, True)
print(game_as_result)

AB pruning player vs Random Player
R R R 
. B B 
. . . 

[(1, 1), (1, 2), (1, 3)] winning disk locations 

* * * 
. B B 
. . . 
1
------------------------------
AB pruning player vs AB pruning player
B B R 
B R R 
. . R 

[(3, 3), (2, 3), (1, 3)] winning disk locations 

B B * 
B R * 
. . * 
1


In [58]:
# Your code here
# hold results for alphabeta vs random players
case_1 = {"alpha": 0, "random":0, "tie": 0}
# holds results for alphabeta vs another alphabeta player
case_2 = {"alpha1": 0, "alpha2":0, "tie": 0}
# now play 10 games
for i in range(10):
    # play the game
    game_run_ar = ConnectFour(3,3,3)
    game_run_aa = ConnectFour(3,3,3)
    # get the results
    game_ar_result = game_run_ar.play_game(alphabeta_player, random_player, False)
    game_aa_result = game_run_aa.play_game(alphabeta_player, alphabeta_player, False)
    # check who wone
    if(game_ar_result == 1):
        case_1["alpha"] += 1
    if(game_ar_result == -1):
        case_1["random"] += 1
    if(game_ar_result == 0):
        case_1["tie"] += 1
    if(game_aa_result == 1):
        case_2["alpha1"] += 1
    if(game_aa_result == -1):
        case_2["alpha2"] += 1
    if(game_aa_result == 0):
        case_2["tie"] += 1

# results of first game
print("Result for alphabeta player vs a random player")
print("Alphabeta player win rate:", case_1["alpha"]/10)
print("Random player win rate:", case_1["random"]/10)
print("Neither player won:", case_1["tie"]/10, "\n")
# results of seccond game
print("Result for alphabeta player vs another alphabeta player")
print("Alphabeta player 1 win rate:", case_2["alpha1"]/10)
print("Alphabeta player 2 win rate:", case_2["alpha2"]/10)
print("Neither player won:", case_2["tie"]/10, "\n")

Result for alphabeta player vs a random player
Alphabeta player win rate: 1.0
Random player win rate: 0.0
Neither player won: 0.0 

Result for alphabeta player vs another alphabeta player
Alphabeta player 1 win rate: 1.0
Alphabeta player 2 win rate: 0.0
Neither player won: 0.0 



`The reason I think AB pruning player vs and another AB pruning player isnt 50/50 is because of the smaller board size. this leads to the advantage of the first player to win!`

### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [52]:
# Your code here
# used reference games4e.py from class Repo for def alpha_beta_search(state, game) function
# used lecture notes as reference same as last alphabeta player but made it to keep track of 
# how many times it went through max_value and min_value but a simple value incrementing per 
# entrence of function. this is so we can compare for effiecency to the minimax function

def alphabeta_player_cost(game):
    # the counter 
    cost = 0
    # defult values for alpha and beta
    alpha = -float('inf')
    beta = float('inf')
    # get best action and value assosicated with it
    value, best_action, cost = max_value_cost(game, game.state, alpha, beta, cost)
    # return the cost
    return cost

# will find the max 
def max_value_cost(game, state, alpha, beta, cost):
    # increment cost
    cost += 1
    # check if game is over
    if game.game_over(state):
        # return no move because game is over and action shouldnt be performed
        return (state.utility, None, cost)
    # starting values 
    temp_value = -float('inf')
    temp_move = None
    # go through all possible moves possible
    for action in state.moves:
        # calculate if new values and moves
        new_value, new_move, cost = min_value_cost(game, game.result(action, state), alpha, beta, cost)
        # get max of values and set the move if the move passed in had a higher value 
        temp_value, temp_move = max(temp_value, new_value), action if new_value > temp_value else temp_move
        # this is the pruning step: we return early and dont check the rest
        if temp_value >= beta:
            return (temp_value, temp_move, cost)
        # update alpha by choosing the max and try again
        alpha = max(temp_value, alpha)
    return (temp_value, temp_move, cost)

# will find the min
def min_value_cost(game, state, alpha, beta, cost):
    # increment cost
    cost += 1
    # check if game is over
    if game.game_over(state):
        # return no move because game is over and action shouldnt be performed
        return (state.utility, None, cost)
    # starting values 
    temp_value = float('inf')
    temp_move = None
    # go through all possible moves possible
    for action in state.moves:
        # calculate if new values and moves
        new_value, new_move, cost = max_value_cost(game, game.result(action, state), alpha, beta, cost)
        # get max of values and set the move if the move passed in had a higher value
        temp_value, temp_move = min(temp_value, new_value), action if new_value < temp_value else temp_move
        # this is the pruning step: we return early and dont check the rest
        if temp_value <= alpha:
            return (temp_value, temp_move, cost)
        # update alpha by choosing the min and try again
        beta = min(temp_value, beta)
    return (temp_value, temp_move, cost)

In [53]:
# Your code here. minimax
# reference from games4e.py class AIMI repo from def minmax_decision(state, game) function  
# same method as alphabeta_player_cost function to compare 
# how many true itturations minimax performes vs the purning method of alphanbeta player 

def minimax_cost(game):        
    # the counter, in this case i have to use a dictionary because the local variable wasnt getting updated
    value_tracker = {"Value": 0, "Tracker": 0}

    def max_value(state):
        # keep track of itterations 
        value_tracker["Tracker"] += 1
        # if games over
        if(game.game_over(state)):
            return state.utility
        else:
            value = -float('inf')
            # instead of pruning, we go through all possibilities
            for action in state.moves:
                value_p = min_value(game.result(action, state))
                if(value_p > value):
                    value = value_p
                    value_tracker["Value"] = value
            return value

    def min_value(state):
        # keep track of itterations 
        value_tracker["Tracker"] += 1
        # if games over
        if(game.game_over(state)):
            return state.utility
        else:
            value = float('inf')
            # instead of pruning, we go through all possibilities
            for action in state.moves:
                value_p = max_value(game.result(action, state))
                if(value_p < value):
                    value = value_p
                    value_tracker["Value"] = value
            return value
    # run and return value
    max_value(game.state)
    return value_tracker["Tracker"]

In [54]:
# run the equation
game_run_counter = ConnectFour(3,3,3)
game_run_counter2 = ConnectFour(3,3,3)
alphabeta_count = alphabeta_player_cost(game_run_counter)
minimax_count = minimax_cost(game_run_counter2)

# result
print("The number of iterations for a Alphabeta calculation is: ", alphabeta_count)
print("The number of iterations for a minimax calculation is: ", minimax_count)
print("The Percentage saving that you will expect when using alphabeta pruning is:", alphabeta_count/minimax_count)

The number of iterations for a Alphabeta calculation is:  18297
The number of iterations for a minimax calculation is:  549946
The Percentage saving that you will expect when using alphabeta pruning is: 0.03327053928931204


`The reasoon why AB pruning algorithm is a lot less computationally heavier than a Minimax alogorithm because it prunes branches that in theory can't affect the final outcome. This reduces the number of itterations need to evaluate for a optimal movement more efficient.`

### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


### (2a) Define a heuristic function
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

**Description of heuristic function**

`This heuristic function takes a board as input and returns a score that represents how good the current board position is for the player whose turn it is. The function checks for horizontal, vertical, and diagonal sequences of 4 (or how ever large the win set is set as) connected pieces and adds points from the score. The more sequences of connected pieces a player has, the higher the score will be. As a reference I used my homework 7 and office hours as help building the idea on how to evaluate a board.`

### (2b) Compare A* to other algorithms
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

In [55]:
# Your code here.
# my heuristic function which is based on my utility function from connect 4
def heuristic_function(game, state):
    # if the game is already over, return 0 as the estimated cost
        if(game.game_over(state)):
            return 0
        # count the number of winning configurations for the player.
        else:
            # used to determin the largest placement value 
            largest_connect = 0
            # create a hypothetical copy of the board, with 'move' executed
            player = state.to_move
            board = cp.deepcopy(state.board)
            num_to_win = game.nwin
            # check for win in direction
            in_a_row = 0
            in_a_col = 0
            in_a_diag1 = 0
            in_a_diag2 = 0

            for r in range(1, game.nrow+1):
                for c in range(1,game.ncol+1):
                    # if spot is where player position was at
                    in_a_row   += board.get((r,c))==player
                    in_a_col   += board.get((r,c))==player
                    in_a_diag1 += board.get((r,c))==player
                    in_a_diag2 += board.get((r,c))==player

                    if(in_a_row == 1): #check for row-wise win
                        in_a_row = 0
                        for x in range(num_to_win):
                            if(board.get((r,c+x)) == player):
                                in_a_row += 1
                            else:
                                break # the breaks are to stop if not sequental
                    if(in_a_col == 1): # check for column-wise win
                        in_a_col = 0
                        for x in range(num_to_win):
                            if(board.get((r-x, c)) == player):
                                in_a_col += 1
                            else:
                                break
                    if(in_a_diag1 == 1): # check left diag win
                        in_a_diag1 = 0
                        for x in range(num_to_win):
                            if(board.get((r-x, c-x)) == player):
                                in_a_diag1 += 1
                            else: 
                                break
                    if(in_a_diag2 == 1): # check right diag win
                        in_a_diag2 = 0
                        for x in range(num_to_win):
                            if(board.get((r-x, c+x)) == player):
                                in_a_diag2 += 1
                            else: 
                                break 

            temp = [in_a_row, in_a_col, in_a_diag1, in_a_diag2]
            # Return the number of winning configurations as the estimated cost.
            largest_connect = max(temp)
        return largest_connect

# the A* search algorithm refrenced from notebook 'obsolete_search4e.ipynb and my Homework 7
def astar_player(game):
    # if the game is already over, return the game's utility value
    if game.game_over(game.state):
        return game.state.utility

    # initialize our queue, path cost, and heauristic cost of path
    queue = []
    path_cost = 0   
    path_heauristic_cost = 0

    # run through each possible action from the current state
    for action in game.state.moves:
        # create a new node using the result of that action
        new_state = game.result(action, game.state)
        # the cost is the number of moves made so far + 1 (which accounts for the move that was used to
        # get to that board) 
        path_cost = len(game.state.moves) + 1
        # now use the heuristic function to get the cost of the current 
        path_heauristic_cost = path_cost + heuristic_function(game, game.state)
        # add the node to the priority que
        heapq.heappush(queue, (path_heauristic_cost, action, new_state))
        
    # if there are still more moves to seach through
    while queue:
        # get the move that have the highest heuristic value cost
        path_heauristic_cost, action, state = heapq.heappop(queue)
        # set the best move to the front of the queue
        best_move = state.moves[0] 
    # now return the best move
    return best_move


In [56]:
# test run against random player
print("A* player VS Random player")
test_game_Astar = ConnectFour(6, 7, 3)
test_game_outcome = test_game_Astar.play_game(astar_player, random_player)
print("---------------------------")
# test run against AB pruning player
print("A* player VS AB pruning player")
game2 = ConnectFour(3, 3, 3)
outcome2 = game2.play_game(astar_player, alphabeta_player, True, True)
    

A* player VS Random player
. . . . R R R 
. . . . . . B 
. . . . . . . 
. . . . . . . 
. . . . . . B 
. . . . . . . 

[(1, 5), (1, 6), (1, 7)] winning disk locations
. . . . * * * 
. . . . . . B 
. . . . . . . 
. . . . . . . 
. . . . . . B 
. . . . . . . 
---------------------------
A* player VS AB pruning player
R B R 
B R B 
. . R 

[(3, 3), (2, 2), (1, 1)] winning disk locations 

* B R 
B * B 
. . * 


In [57]:
# now run Test
count1 = 0
count2 = 0
# Play 10 games of Connect Four using the A* player and a random player
for i in range(10):
    # Create the new Connect Four game
    game = ConnectFour(6, 7, 4)
    game2 = ConnectFour(3, 3, 3)
    # play both games 
    outcome = game.play_game(astar_player, random_player, False)
    outcome2 = game2.play_game(astar_player, alphabeta_player, False)
    # check outcome 
    if (outcome == 1):
        count1+=1
        #print("The A* player won!")
    if (outcome2 == 1):
        count2+=1

# compare results
print()
print("A* vs random player won", count1, "out of 10 game")
print("A* vs AB pruing player won", count2, "out of 10 game")


A* vs random player won 9 out of 10 game
A* vs AB pruing player won 10 out of 10 game


**Compare results** 

`In comparison to the random player and the AB pruning player, the A* player is likely to perform better because it to goes though more runes to determine the best move to make in order to win if our heuristic value is calculated right. It is also more efficient than the minimax player because it uses the estimated cost of reaching the goal state (which was the number of planks needed to be placed in a row to win) from any given state (board after a move was made). Making A* win more games.`

**Did one player win more than the other?**

`When playing 10 games of Connect Four using the A* player and the random player, the A* player most of the time won all game but at times the random player won some games. This is due to the fact that when the A* player calculates a move, the random player could accidently block A* or could acidently place a plank in a winning spot. In the games against the AB pruning player, the A* player all games against the AB pruning player. This is due to the fact that A* is more indepth algorith while AB pruning shortnes the time but could miss a critiqal move. `

**How often was the game a draw?**

`There were no draws in these games either. Majority of the time, A* is the best player.`


**How many moves did each player make?**

`The A* player was able to make more effective use of the heuristic function to guide the search towards potential winning positions and make better decisions, but since A* uses a modified minimax algorithm -> as we saw with question 1e it is the most computationally heavy then AB pruning and random player. This leads to number of moves in accending order being: Random_player, AB pruning, A* player.`

**Were there situations where one player appeared to do better than the other?**

`while there were some situations where the Random player appeared to do better than the A* player, this doesnt say that A* isnt as amazing as it is. Random player did get lucky at time but not majority of the times. The only classification I would say AB pruning or Random player do better than A* is the performance asspect of the code.`


**Given the outcome, are there other heuristics you would like to implement?**

`I would love to be able to implement a heauristc that could make A* faster.`